<a href="https://colab.research.google.com/github/KayKozaronek/03_Courses/blob/master/Copia_de_Cats_and_Dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this exercise a CNN is applied on the **full** Cats vs. Dogs dataset (25000 images)
 

In [0]:
import os 
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd

This code block downloads the full dataset and stores it as `cats-and-dogs.zip`. 

It then unzips it to /tmp which will create a tmp/PetImages directory containing subdirectories called "Cat" and "Dog"

If the URL doesn't work, 
visit https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765 and right click on the 'Download Manually' link to get a new URL

In [2]:
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = "/tmp/cats-and-dogs.zip"
zip_ref = zipfile.ZipFile(local_zip,"r")

zip_ref.extractall("/tmp")
zip_ref.close()


--2020-04-28 09:14:45--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.35.72.22, 2600:1407:a800:192::e59, 2600:1407:a800:18d::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.35.72.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.68M   200MB/s    in 4.1s    

2020-04-28 09:14:50 (192 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824894548/824894548]



Look at how many pictures there are in each directory
Expected values:
- 12501
- 12501

In [3]:
print(len(os.listdir("/tmp/PetImages/Cat/")))
print(len(os.listdir("/tmp/PetImages/Dog/")))

12501
12501


Use `os.mkdir` to create your directories.

You will need a **directory** for cats-v-dogs, and **subdirectories** for training and testing. 

These in turn will need **subdirectories** for "cats" and "dogs"


In [4]:
help(os.mkdir)

Help on built-in function mkdir in module posix:

mkdir(path, mode=511, *, dir_fd=None)
    Create a directory.
    
    If dir_fd is not None, it should be a file descriptor open to a directory,
      and path should be relative; path will then be relative to that directory.
    dir_fd may not be implemented on your platform.
      If it is unavailable, using it will raise a NotImplementedError.
    
    The mode argument is ignored on Windows.



In [0]:
try:
  os.makedirs("/tmp/cats-v-dogs/training/cats")
  os.makedirs("/tmp/cats-v-dogs/training/dogs")
  os.makedirs("/tmp/cats-v-dogs/testing/cats")
  os.makedirs("/tmp/cats-v-dogs/testing/dogs")
except OSError:
  pass

Write a **Python function** called `split_data` which takes 
- a **SOURCE directory** containing the files 
- a **TRAINING directory** that a portion of the files will be copied to
- a **TESTING directory** that a portion of the files will be copie to
- a **SPLIT SIZE** to determine the portion.


The files should also be **randomized**, so that the training set is a random X% of the files, and the test set is the remaining files.

So, for example, if SOURCE is PetImages/Cat, and SPLIT SIZE is .9, then 90% of the images in PetImages/Cat will be copied to the TRAINING dir and 10% of the images will be copied to the TESTING dir.

Also -- All images should be checked, and if they have a zero file length, they will not be copied over 


- `os.listdir(DIRECTORY)` gives you a listing of the contents of that directory

- `os.path.getsize(PATH)` gives you the size of the file 

- `copyfile(source, destination)` copies a file from source to destination

- `random.sample(list, len(list))` shuffles a list

In [6]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
  # YOUR CODE STARTS HERE
  files =[]
  for filename in os.listdir(SOURCE):
    file = SOURCE + filename 
    if os.path.getsize(file) > 0:
      files.append(filename)
    else:
      print(filename + " is zero length, so ignoring.")

  training_length = int(len(files)* SPLIT_SIZE)
  testing_length = int(len(files) - training_length)
  shuffled_set = random.sample(files, len(files))
  training_set = shuffled_set[0:training_length]
  testing_set = shuffled_set[-testing_length:]

  for filename in training_set:
    this_file = SOURCE + filename
    destination = TRAINING + filename 
    copyfile(this_file , destination)

  for filename in testing_set:
    this_file = SOURCE + filename 
    destination = TESTING + filename 
    copyfile(this_file, destination)
  # YOUR CODE ENDS HERE


CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

# Expected output
# 666.jpg is zero length, so ignoring
# 11702.jpg is zero length, so ignoring

666.jpg is zero length, so ignoring.
11702.jpg is zero length, so ignoring.


In [7]:
print(len(os.listdir("/tmp/cats-v-dogs/training/cats/")))
print(len(os.listdir('/tmp/cats-v-dogs/training/dogs/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/dogs/')))

# Expected output:
# 11250
# 11250
# 1250
# 1250

11250
11250
1250
1250


Define a Keras model to classify cats vs. dogs
uas at least 3 convolution layers 

In [0]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16,(3,3), input_shape=(150,150,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer=RMSprop(lr=0.001), 
              loss ="binary_crossentropy",
              metrics =["accuracy"])

In [9]:
TRAINING_DIR = "/tmp/cats-v-dogs/training/"
train_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=100,
                                                    class_mode="binary",
                                                    target_size=(150,150))

VALIDATION_DIR = "/tmp/cats-v-dogs/testing/"
validation_datagen = ImageDataGenerator(rescale= 1.0/255.0)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size = 100,
                                                              class_mode ="binary",
                                                              target_size =(150,150))




# Expected Output:
# Found 22498 images belonging to 2 classes.
# Found 2500 images belonging to 2 classes.

Found 22498 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


The expectation here is that the model will train, and that accuracy will be > 95% on both training and validation

i.e. acc:A1 and val_acc:A2 will be visible, and both A1 and A2 will be > .9

In [0]:
history = model.fit(train_generator, 
                    epochs=15,
                    verbose=1,
                    validation_data =validation_generator)

Epoch 1/15
    132/Unknown - 276s 2s/step - loss: 0.8255 - accuracy: 0.5914

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

   1450/Unknown - 3037s 2s/step - loss: 0.4242 - accuracy: 0.8008

Plot Loss and accuracy

In [0]:
%matplotlib inline 

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------

acc=history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss=history.history["loss"]
val_loss = history.history["val_los"]

epochs = range(len(acc)) #Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------

plt.plot(epochs,acc,"r","Training Accuracy")
plt.plot(epochs, val_acc, "b", "Validation Accuracy")
plt.title("Training and validation accuracy")
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------

plt.plot(epochs, loss, "r", "Training Loss")
plt.plot(epochs, vall_loss, "b", "Validation Loss")

plt.title("Training and validation loss")

# Desired output. Charts with training and validation metrics. No crash 

Here's a codeblock just for fun. You should be able to upload an image here 
and have it classified without crashing

In [0]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():

  # Predicting images
  path = "/content/" +fn
  img = image.load_img(path, target_size=(#Your Code Here))
  x = image.img_to_array(img)
  x = np.expand_dims(x,axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + "is a dog")
  else:
    print(fn + "is a cat")